In [ ]:
# 008/ClassificationRegression-11si/Classification/IRISDatasetDecisionTree.ipynb

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import Binarizer
import numpy as np
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as stats
from binance.enums import HistoricalKlinesType
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from main_funcs import get_and_update_data


In [ ]:
df_source = get_and_update_data("BTCUSDT", "4h", HistoricalKlinesType.SPOT,country=None)
df_source = df_source[["open", "high", "low", "close","volume" ]]
# df_source = df_source.iloc[:-4,:]
df_source

In [ ]:
df = df_source.copy()
# df = df.iloc[9000:,:]
df["open_close"] = df.close - df.open
df["open_close_percent"] = df.open_close / df.open
period = 60
for i in range(1,period+1):
    df[f"open_close_percent_{i}"] = df.shift(i).open_close_percent

darsad_target=0.0
df["target"] = df.shift(-1).open_close_percent
df["target"] = np.where(df['target'] > darsad_target, 1, np.where(df['target'] < -darsad_target, -1, 0))
# df["target2"] = np.where( (df['target'] <  0.02) and (df['target'] > -0.02),0, 0 )
# df.dropna(inplace=True)
df.dropna(subset=[f'open_close_percent_{period}'],inplace=True)
df.drop(columns=['open','high','low','close','volume','open_close'],inplace=True)

df

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# model

# all ok rows and columns
x_ok = df.iloc[:-1,:-1]
y_ok = df.iloc[:-1,-1:]
# fit data count
num = int(len(df)*0.7)
# fit data
x = x_ok[:num]
y = y_ok[:num]
# fit validator
X = x_ok[num:]
Y = y_ok[num:]



knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(x.to_numpy(), y.to_numpy().ravel())


In [ ]:
# print(df.iloc[-1:,:-1])
# svm_classifier.predict(df.iloc[-1:,:-1])

In [ ]:
Y_RESULT = knn_clf.predict(X.to_numpy())

In [ ]:
print(metrics.classification_report(Y,Y_RESULT))

In [ ]:
cm = metrics.confusion_matrix(Y, Y_RESULT, labels=knn_clf.classes_)
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn_clf.classes_)
disp.plot()
plt.show()